In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cpu


In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [3]:
# MLP model for classification
model = nn.Sequential(
    nn.Linear(4, 16),  # Input layer (4 features to 16 neurons)
    nn.ReLU(),
    nn.Linear(16, 3)   # Output layer (16 neurons to 3 classes)
)

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/50], Loss: 1.0966
Epoch [2/50], Loss: 1.0687
Epoch [3/50], Loss: 1.0388
Epoch [4/50], Loss: 1.0050
Epoch [5/50], Loss: 0.9697
Epoch [6/50], Loss: 0.9510
Epoch [7/50], Loss: 0.9125
Epoch [8/50], Loss: 0.8743
Epoch [9/50], Loss: 0.8421
Epoch [10/50], Loss: 0.8344
Epoch [11/50], Loss: 0.7967
Epoch [12/50], Loss: 0.7855
Epoch [13/50], Loss: 0.7444
Epoch [14/50], Loss: 0.7088
Epoch [15/50], Loss: 0.6969
Epoch [16/50], Loss: 0.6488
Epoch [17/50], Loss: 0.6473
Epoch [18/50], Loss: 0.6159
Epoch [19/50], Loss: 0.5955
Epoch [20/50], Loss: 0.5825
Epoch [21/50], Loss: 0.5616
Epoch [22/50], Loss: 0.5413
Epoch [23/50], Loss: 0.5390
Epoch [24/50], Loss: 0.5250
Epoch [25/50], Loss: 0.5053
Epoch [26/50], Loss: 0.5066
Epoch [27/50], Loss: 0.4943
Epoch [28/50], Loss: 0.4892
Epoch [29/50], Loss: 0.4724
Epoch [30/50], Loss: 0.4611
Epoch [31/50], Loss: 0.4501
Epoch [32/50], Loss: 0.4435
Epoch [33/50], Loss: 0.4335
Epoch [34/50], Loss: 0.4255
Epoch [35/50], Loss: 0.4234
Epoch [36/50], Loss: 0.4115
E

In [4]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test dataset: {100 * correct / total:.2f}%')


Accuracy of the model on the test dataset: 93.33%


In [6]:
torch.save(model.state_dict(), 'iris_classification_model.pth')